# 01 Data preprocessing

- 이 단계에서는 데이터 셋을 다운받고, Amazon Translate을 사용해 한국어로 번역한 후 Amazon S3에 저장하는 과정을 수행합니다.

### 데이터셋 불러오기

In [1]:
from datasets import load_dataset

# 데이터셋 불러오기
dataset = load_dataset("b-mc2/sql-create-context")

# 'train' split에서 첫 번째 샘플 출력
print("전문: ", dataset['train'][0])
print("#######구분해서 보기######")
sample = dataset['train'][0]
print("Answer:", sample['answer'])
print("Question:", sample['question'])
print("Context:", sample['context'])

ModuleNotFoundError: No module named 'datasets'

### Amazon Translate으로 데이터 한국어로 번역하기

In [ ]:
from datasets import load_dataset
import boto3

# AWS 클라이언트 생성
translate = boto3.client(service_name='translate', region_name='us-east-1')

# 데이터셋 불러오기
dataset = load_dataset("b-mc2/sql-create-context")

# 'question' 열 추출 및 번역
translated_dataset = dataset.map(
    lambda sample: {
        'answer': sample['answer'],
        'question': [
            translate.translate_text(Text=question, SourceLanguageCode="en", TargetLanguageCode="ko").get('TranslatedText')
            for question in sample['question']
        ],
        'context': sample['context']
    },
    batched=True,
    desc="Translating questions to Korean"
)

# 번역된 데이터셋 저장
translated_dataset.save_to_disk('data/ko_train_data/train')